In [65]:
from operator import attrgetter
from sortedcontainers import SortedList
import bisect
import math
import random as random
from pyskiplist import SkipList

In [66]:
def binarysearch(a, x):
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        #print(a[i], x)
        return True
    else:
        return False

In [67]:
class MinMaxList:
    def __init__(self):
        self.nodes = []
        self.min = float("inf")
        self.max = float("-inf")

    def __lt__(self, other):
        return self.min < other

class TeleportList:

    def __init__(self):
        self.height = -1
        self.blists = []

    def lookup(self, nr):
        for he in self.blists:
            if nr < he.min:
                break
            else:
                if he.max >= nr:
                    if binarysearch(he.nodes, nr):
                        return True
        return False

    def findsert(self, a, x):
        L = len(a)
        i = bisect.bisect_left(a, x)
        if i != L and a[i] == x:
            return False
        else:
            return i + 1

    def insert(self, nr):
        height = int(-(math.log2(random.random())))
        if self.height < height:
            for i in range(height - self.height):
                self.blists.append(MinMaxList())
            self.height = height

        test = self.blists[height]

        i = self.findsert(test.nodes, nr)

        if i is not False:
            test.nodes.insert(i - 1, nr)
            if test.min > nr:
                test.min = nr
            if test.max < nr:
                test.max = nr
            self.blists.sort(key=attrgetter('min'))
        else:
            return False

    def show_hedges(self):
        for i in self.blists:
            print(len(i.nodes))

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

In [85]:
#splits the overloaded list into two consequtive parts
def splitter(arr, load):
    half = load // 2 
    zs = [0] * half
    for i in range(half-1, -1, -1):
        zs[i] = arr.pop()
    return zs

# if overload is detected, splits and adds a new split into levellist
def Overload(blist, i, load):
    B = IntervalList(5)
    candidate_sublist = blist.sublists[i]
    B.nodes = splitter(candidate_sublist.nodes, load)
    #B.i = - blist.sublists[i].i - 1
    B.max = candidate_sublist.max
    candidate_sublist.max = candidate_sublist.nodes[-1]
    blist.sublists.insert(i+1, B)

class IntervalList:
    def __init__(self, nr):
        self.nodes = [nr]
        self.max = float("-inf")

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class LevelList:
    def __init__(self):
        self.sublists = []
        self.min = float("inf")
        self.max = float("-inf")

    def __lt__(self, other):
        return self.max > other

class NovusList:# Rucy, rename it!
    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 2000


    def lookup(self, nr):

        for he in self.blists:
            
            if nr <= he.sublists[-1].max:
                i = bisect.bisect_left(he.sublists, nr)
                
                if  i != len(he.sublists) and he.sublists[i].nodes[0] <= nr:
                    if binarysearch(he.sublists[i].nodes, nr):
                        return True
        #print(nr) #can be used to detect if the search works or not
        return nr

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                B = LevelList()
                C = IntervalList(nr)
                C.max = -1 #arbitrary contemporary max
                B.sublists.append(C)
                self.blists.append(B)
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height]
        
        ## Doing the search to see which Intervallist it should be in
        L = len(blist.sublists)
        i = bisect.bisect_left(blist.sublists, nr)
        
        ## If it's smaller than all other elements then just insort it
        if i == 0 or L == 1:
            candid = blist.sublists[0]
            bisect.insort_left(candid.nodes, nr)
            
            candid.max = candid.nodes[-1]
            
            if len(candid.nodes) == self.load:
                    Overload(blist, 0, self.load)
        ## If it's bigger than all the other elements than just append it
        elif i == L:
            candid = blist.sublists[-1]
            candid.nodes.append(nr)
            candid.max = nr
            
            if len(candid.nodes) == self.load:
                    Overload(blist, i-1, self.load)

            ## Else add it
        else:
            candidate_sublist = blist.sublists[i]
            # if the element is also bigger than the minimum of the current list than we insort it
            if candidate_sublist.nodes[0] <= nr:
                bisect.insort_left(candidate_sublist.nodes, nr)
                
                if len(blist.sublists[i].nodes) == self.load:
                    Overload(blist, i, self.load)
            # then the element must be smaller then the min of the current list but therefore
            # bigger than the max of the previous list-- so we just append it
            else:
                candidate_sublist = blist.sublists[i-1]
                candidate_sublist.nodes.append(nr)
                candidate_sublist.max = nr
            
                if len(candidate_sublist.nodes) == self.load:
                    Overload(blist, i-1, self.load)
            
            

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)
            
    def show_edges(self):
        for i in self.blists:
            print("--------" + str(len(i.sublists)) +"-----------")
            for j in i.sublists:
                print(j.nodes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

In [102]:
#random.seed(0)
hgraph = TeleportList()
skeepFast = SkipList()
slist = SortedList()
nlist = NovusList()


nr = 100000

ten_thousand_integers = [random.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.insert(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        sl.add(ten_thousand_integers[i])

def ten_k_skeep(skeep):
    for i in range(nr):
        skeep.insert(ten_thousand_integers[i], ten_thousand_integers[i])
        
def ten_k_novus(novus):
    for i in range(nr):
        novus.insert(ten_thousand_integers[i])
        
%timeit -r 1 -n 1 ten_k_novus(nlist)       
%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(slist)
%timeit -r 1 -n 1 ten_k_skeep(skeepFast)


#nlist.show_edges()
#print(np.bincount(nlist.bin))
#print(np.bincount(nlist.antis))
#nlist.show_hedges()

## novus & fixed teleport in this notebook
## delete (mark or delete)
## optimization 
## push
## complexity estimation
# paper

1 loop, best of 1: 430 ms per loop
1 loop, best of 1: 817 ms per loop
1 loop, best of 1: 337 ms per loop
1 loop, best of 1: 2.45 s per loop


In [103]:
#random.seed(0)
#nr = 100000
#ten_thousand_integers = [rand.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

def ten_k_skeep(skeep):
    for i in range(nr):
        ten_thousand_integers[i] in skeep
        
def ten_k_novus(novus):
    for i in range(nr):
        novus.lookup(ten_thousand_integers[i])
        
%timeit -r 1 -n 1 ten_k_novus(nlist) 
%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(slist)
%timeit -r 1 -n 1 ten_k_skeep(skeepFast)

1 loop, best of 1: 681 ms per loop
1 loop, best of 1: 423 ms per loop
1 loop, best of 1: 190 ms per loop
1 loop, best of 1: 1.29 s per loop


In [104]:
#random.seed(1)
nr = 100000
ten_thousand_integers = [random.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

def ten_k_skeep(skeep):
    for i in range(nr):
        ten_thousand_integers[i] in skeep
        
def ten_k_novus(novus):
    for i in range(nr):
        novus.lookup(ten_thousand_integers[i]) 
            
        
%timeit -r 1 -n 1 ten_k_novus(nlist) 
%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(slist)
%timeit -r 1 -n 1 ten_k_skeep(skeepFast)

1 loop, best of 1: 2.87 s per loop
1 loop, best of 1: 1.47 s per loop
1 loop, best of 1: 179 ms per loop
1 loop, best of 1: 1.22 s per loop
